# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: CU registers UE and relies on RRC Setup Success when receiving initialRRCTransfer with mutated DU ID and Transaction ID before F1 Setup
Repro Steps:
1) Attacker sends an incorrect UE Context Setup Request to the gNB-CU.
2) gNB-CU rejects the request as expected.
3) Attacker immediately sends a UE Context Setup Success.
4) In parallel, attacker sends an initialRRCTransfer with mutated DU ID and Transaction ID.
5) Attacker follows with an RRC Setup Success message.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "CU accepts attacker-driven message flow: after rejecting an invalid UE Context Setup Request, CU later registers UE when receiving Initial UL RRC Message Transfer carrying mutated gNB-DU UE F1AP ID and Transaction ID, followed by RRCSetupComplete, allegedly even before F1 Setup completes.",
    "interface_guess": ["F1AP", "RRC"],
    "procedure_guess": [
      "F1 Setup",
      "Initial UL RRC Message Transfer",
      "UL/DL RRC Message Transfer",
      "UE Context Setup"
    ],
    "components_involved": ["gNB-CU", "gNB-DU", "UE"],
    "key_ids": {
      "transaction_id": "mutated (attacker-controlled)",
      "cu_ue_f1ap_id": "unknown",
      "du_ue_f1ap_id": "mutated (attacker-controlled)",
      "du_id": "unknown/attacker-claimed",
      "rnti": "unknown",
      "pci": "unknown",
      "served_cell_list": []
    },
    "signals_or_timers": [
      "F1AP.TransactionID",
      "F1AP.gNB-DU UE F1AP ID",
      "F1AP.gNB-CU UE F1AP ID",
      "Initial UL RRC Message Transfer",
      "UE Context Setup (Req/Resp)",
      "RRCSetupComplete"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": ["Initial UL RRC", "non-UE-associated", "establish UE-associated"],
      "title": ["§8.4.1 Initial UL RRC Message Transfer", "§8.4.1.2 Successful operation", "§9.2.3.1 INITIAL UL RRC MESSAGE TRANSFER"]
    },
    {
      "Keywords": ["UE Context Setup", "Req/Resp pairing", "directionality"],
      "title": ["§8.3 UE CONTEXT SETUP procedure", "§9.2.2.3 UE CONTEXT SETUP FAILURE"]
    },
    {
      "Keywords": ["Transaction ID", "procedure scoping", "initiating peer"],
      "title": ["§9.3.1.23 Transaction ID"]
    },
    {
      "Keywords": ["UE identifiers", "binding", "CU/DU UE F1AP ID"],
      "title": ["§9.3.1.4 gNB-CU UE F1AP ID", "§9.3.1.5 gNB-DU UE F1AP ID"]
    },
    {
      "Keywords": ["F1 Setup", "precondition", "non-UE signaling"],
      "title": ["§9.2.1 F1 SETUP (incl. §9.2.1.6 Failure)"]
    }
  ],
  "expected_behaviour": "DU sends Initial UL RRC Message Transfer (non-UE-associated) with mandatory NRCGI, C-RNTI, RRC-Container, Transaction ID, and gNB-DU UE F1AP ID; CU initiates establishment of the UE-associated logical F1-connection and validates identifier bindings. UE Context Setup is CU→DU Request and DU→CU Response; unsolicited or wrongly directed messages must be rejected. Transaction ID uniquely scopes a procedure and is set by the initiating peer; mismatched or replayed IDs must not be accepted."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "function_name": "CU_handle_INITIAL_UL_RRC_MESSAGE_TRANSFER",
      "reason": "CU-side handler for Initial UL RRC Message Transfer; entry point after DU sends initial UL RRC (non-UE-associated)."
    },
    {
      "path": "F1AP/lib/f1ap_rrc_message_transfer.c",
      "function_name": "decode_initial_ul_rrc_message_transfer",
      "reason": "Decodes mandatory fields (gNB-DU UE F1AP ID, NRCGI, C-RNTI, RRCContainer, Transaction ID)."
    },
    {
      "path": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "function_name": "CU_handle_UL_RRC_MESSAGE_TRANSFER",
      "reason": "Performs UE ID binding check (CU UE ID exists; DU UE ID matches)."
    },
    {
      "path": "F1AP/f1ap_cu_ue_context_management.c",
      "function_name": "CU_handle_UE_CONTEXT_SETUP_RESPONSE",
      "reason": "Processes UE Context Setup Response in CU, pairs with CU-initiated Request."
    },
    {
      "path": "F1AP/f1ap_du_ue_context_management.c",
      "function_name": "DU_handle_UE_CONTEXT_SETUP_REQUEST",
      "reason": "Directionality: DU handles Request from CU; rejects invalid directions."
    },
    {
      "path": "F1AP/f1ap_cu_interface_management.c",
      "function_name": "CU_handle_F1_SETUP_REQUEST",
      "reason": "F1 Setup on CU side; prerequisite non-UE-associated control plane."
    },
    {
      "path": "F1AP/f1ap_handlers.c",
      "function_name": "handlers array (InitialULRRCMessageTransfer, UEContextSetup)",
      "reason": "Procedure registration and mapping to handlers for directionality."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§8.4.1.2 Successful operation",
      "text": "The gNB-DU initiates the procedure by sending an INITIAL UL RRC MESSAGE TRANSFER. The establishment of the UE-associated logical F1-connection shall be initiated as part of the procedure. If the RRC-Container-RRCSetupComplete IE is included, the gNB-CU shall take it into account as specified in TS 38.401."
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§9.2.3.1 INITIAL UL RRC MESSAGE TRANSFER",
      "text": "Direction: gNB-DU → gNB-CU. Mandatory IEs: gNB-DU UE F1AP ID, NR CGI, C-RNTI, RRC-Container, Transaction ID. Optional: DU to CU RRC Container, RRC-Container-RRCSetupComplete, …"
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§9.3.1.23 Transaction ID",
      "text": "Uniquely identifies a procedure among all ongoing parallel procedures of the same type initiated by the same protocol peer. Messages belonging to the same procedure use the same Transaction ID. The Transaction ID is determined by the initiating peer."
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§8.3 UE CONTEXT SETUP / §9.2.2.3 Failure",
      "text": "UE CONTEXT SETUP is CU→DU Request and DU→CU Response/Failure. Messages include gNB-CU UE F1AP ID and optionally gNB-DU UE F1AP ID; wrong direction or unknown IDs lead to failure."
    },
    {
      "kind": "code",
      "source": "F1AP/lib/f1ap_rrc_message_transfer.c",
      "location": "97-104",
      "text": "// c6. Transaction ID (integer value) ... ie6->id = ..._TransactionID; ie6->value.choice.TransactionID = msg->transaction_id;"
    },
    {
      "kind": "code",
      "source": "F1AP/lib/f1ap_rrc_message_transfer.c",
      "location": "120-131,137-139",
      "text": "decode_initial_ul_rrc_message_transfer: extracts gNB_DU_ue_id, NRCGI, C_RNTI, RRCContainer; then TransactionID → out->transaction_id;"
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "location": "114-128",
      "text": "CU_handle_UL_RRC_MESSAGE_TRANSFER: validates that CU UE ID exists and DU UE ID matches stored mapping; otherwise logs error and returns -1."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "location": "48-61",
      "text": "CU_handle_INITIAL_UL_RRC_MESSAGE_TRANSFER: decodes Initial UL RRC, creates ITTI F1AP_INITIAL_UL_RRC_MESSAGE, forwards to RRC without additional checks."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_handlers.c",
      "location": "53-61",
      "text": "handlers table includes UEContextSetup (DU_handle_UE_CONTEXT_SETUP_REQUEST, CU_handle_UE_CONTEXT_SETUP_RESPONSE) and InitialULRRCMessageTransfer (CU_handle_INITIAL_UL_RRC_MESSAGE_TRANSFER)."
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-DU", "to": "gNB-CU", "message": "INITIAL UL RRC MESSAGE TRANSFER (NR CGI, C-RNTI, RRC-Container, Transaction ID)", "precond": "F1 Setup established", "postcond": "CU initiates UE-associated F1 logical connection"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT SETUP REQUEST", "precond": "Initial UL RRC processed", "postcond": "DU configures UE"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT SETUP RESPONSE", "precond": "DU config done", "postcond": "UE context active"},
    {"from": "UE", "to": "gNB-DU→gNB-CU", "message": "RRCSetupComplete (via UL/DCCH or Initial UL RRC container)", "precond": "RRCSetup delivered", "postcond": "Registration info consumed"}
  ],
  "state_machine": {
    "states": [
      "F1_NOT_SETUP",
      "F1_SETUP",
      "AWAIT_INITIAL_UL_RRC",
      "UE_CTX_SETUP_REQ_SENT",
      "UE_CTX_ACTIVE",
      "ERROR"
    ],
    "transitions": [
      {"from": "F1_NOT_SETUP", "to": "F1_SETUP", "on": "F1 SETUP complete"},
      {"from": "F1_SETUP", "to": "AWAIT_INITIAL_UL_RRC", "on": "Ready for access"},
      {"from": "AWAIT_INITIAL_UL_RRC", "to": "UE_CTX_SETUP_REQ_SENT", "on": "Initial UL RRC (valid)", "guard": "Transaction ID present; gNB-DU UE F1AP ID well-formed"},
      {"from": "UE_CTX_SETUP_REQ_SENT", "to": "UE_CTX_ACTIVE", "on": "UE CONTEXT SETUP RESPONSE", "guard": "IDs pair (CU/DU UE F1AP IDs consistent)"},
      {"from": "AWAIT_INITIAL_UL_RRC", "to": "ERROR", "on": "Invalid direction UE CONTEXT SETUP (from attacker)", "guard": "reject"},
      {"from": "F1_NOT_SETUP", "to": "ERROR", "on": "Any UE-associated msg", "guard": "reject until F1 Setup"}
    ]
  },
  "assumptions": [
    "F1 Setup must complete before processing Initial UL RRC",
    "CU validates Transaction ID scoping and ID bindings on all UE-associated procedures",
    "Unsolicited or misdirected UE Context messages are rejected"
  ],
  "questions": [
    "Where in CU code is F1 Setup completion checked before accepting Initial UL RRC?",
    "Is Transaction ID tracked across Initial UL RRC and subsequent UE Context procedure?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "Directionality of UE Context Setup (CU→DU Req, DU→CU Resp)",
      "result": "FAIL",
      "evidence": [
        "Spec: F1AP.docx §8.3 (UE Context Setup) / §9.2.2.3 Failure",
        "Code: F1AP/f1ap_handlers.c 53-61 (handler bindings show direction)"
      ]
    },
    {
      "name": "Initial UL RRC MT requires Transaction ID and DU UE ID",
      "result": "PASS",
      "evidence": [
        "Spec: F1AP.docx §9.2.3.1 (Transaction ID mandatory)",
        "Code: F1AP/lib/f1ap_rrc_message_transfer.c 97-104,120-131,137-139"
      ]
    },
    {
      "name": "CU validates ID binding on subsequent UL RRC MT",
      "result": "PASS",
      "evidence": [
        "Code: F1AP/f1ap_cu_rrc_message_transfer.c 114-128 (CU UE ID exists; DU UE ID matches)"
      ]
    },
    {
      "name": "Reject unsolicited/misdirected UE Context messages",
      "result": "UNKNOWN",
      "evidence": [
        "Spec: F1AP.docx §8.3 directionality implies rejection",
        "Code: No explicit CU-side guard shown for misdirected Success prior to Request"
      ]
    },
    {
      "name": "Precondition: F1 Setup before UE-associated procedures",
      "result": "UNKNOWN",
      "evidence": [
        "Spec: F1AP.docx §8.4.1.2 (Init UL RRC triggers UE-assoc connection)",
        "Spec: §9.2.1 F1 SETUP (not explicitly cross-checked here)"
      ]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "Spec mandates correct directionality and scoped Transaction ID. Code decodes and forwards Initial UL RRC without additional validation and later enforces ID binding only for UL RRC Message Transfer, leaving a window where attacker-crafted Initial UL RRC with mutated Transaction ID/DU UE ID plus misdirected UE Context messages could be acted upon, leading to premature UE registration. Additional CU-side guards should reject misdirected UE Context messages and enforce F1 Setup/transaction scoping before accepting RRCSetupComplete."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "F1AP.docx §8.4.1.2 Successful operation (Initial UL RRC Message Transfer)",
      "F1AP.docx §9.2.3.1 INITIAL UL RRC MESSAGE TRANSFER (IEs incl. Transaction ID)",
      "F1AP.docx §9.3.1.23 Transaction ID (procedure scoping, initiating peer)",
      "F1AP.docx §8.3 UE CONTEXT SETUP / §9.2.2.3 Failure (direction, IDs)",
      "F1AP.docx §9.2.1 F1 SETUP (context for initialisation)"
    ],
    "code_citations": [
      "F1AP/lib/f1ap_rrc_message_transfer.c:97-104 (encode Transaction ID)",
      "F1AP/lib/f1ap_rrc_message_transfer.c:120-131,137-139 (decode IDs + Transaction ID)",
      "F1AP/f1ap_cu_rrc_message_transfer.c:48-61 (handle Initial UL RRC → ITTI)",
      "F1AP/f1ap_cu_rrc_message_transfer.c:114-128 (validate CU/DU UE ID binding on UL RRC MT)",
      "F1AP/f1ap_handlers.c:53-61 (UE Context Setup, Initial UL RRC handler mapping)"
    ],
    "expected_vs_observed": "Expected: CU only registers UE following a valid Initial UL RRC from DU, then CU→DU UE Context Setup Request and DU→CU Response, using consistent UE F1AP IDs and Transaction ID scoping. Observed: CU appears to accept attacker-sent Initial UL RRC with mutated IDs and misdirected UE Context messages, relying on RRCSetupComplete to register prior to or without proper F1-side validation."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "Spec requires Initial UL RRC to originate from DU, with Transaction ID scoping and consistent UE F1AP IDs; UE Context Setup is CU→DU (Req) and DU→CU (Resp). Code forwards Initial UL RRC to RRC without CU-side validation and only enforces ID binding later for UL RRC. If the system relies on RRCSetupComplete carried in Initial UL RRC to register a UE without verifying F1-side transaction/ID context and directionality, an attacker can spoof DU UE ID/Transaction ID and misdirect UE Context messages to trigger registration.",
    "citations": {
      "spec": [
        "F1AP.docx §8.4.1.2",
        "F1AP.docx §9.2.3.1",
        "F1AP.docx §9.3.1.23",
        "F1AP.docx §8.3 / §9.2.2.3",
        "F1AP.docx §9.2.1"
      ],
      "code": [
        "F1AP/lib/f1ap_rrc_message_transfer.c:97-104",
        "F1AP/lib/f1ap_rrc_message_transfer.c:120-131,137-139",
        "F1AP/f1ap_cu_rrc_message_transfer.c:48-61",
        "F1AP/f1ap_cu_rrc_message_transfer.c:114-128",
        "F1AP/f1ap_handlers.c:53-61"
      ]
    },
    "suggested_repro": [
      "Bring up CU and block/skip F1 Setup, then send Initial UL RRC with crafted DU UE ID + Transaction ID to CU SCTP endpoint.",
      "Send UE CONTEXT SETUP REQUEST (invalid direction) to CU; confirm rejection.",
      "In parallel, send UE CONTEXT SETUP SUCCESS (invalid direction) and an Initial UL RRC embedding RRCSetupComplete; observe whether CU registers UE."
    ],
    "risks": [
      "UE spoofing/registration without DU involvement",
      "State desynchronization between CU and DU",
      "Potential resource leakage or security policy bypass"
    ],
    "next_steps": [
      "Add CU-side guard: ignore UE CONTEXT messages arriving in the wrong direction or without an outstanding Request.",
      "Track and validate F1AP Transaction ID across Initial UL RRC and subsequent UE-associated procedures at CU.",
      "Gate acceptance of RRCSetupComplete until UE Context Setup completes and CU/DU UE F1AP IDs are bound.",
      "Add unit/integration tests covering invalid direction and mutated IDs."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.